In [1]:
import polars as pl

# Load cleaned data
cleaned_data = pl.read_parquet('/path/to/cleaned_train.parquet')

# Load lagged data
lagged_data = pl.read_parquet('/path/to/lags.parquet')

# Merge based on the 'date_id' column
combined_data = cleaned_data.join(lagged_data, on='date_id', how='left')


ModuleNotFoundError: No module named 'polars'

In [ ]:
lagged_data = lagged_data.fill_null(strategy="mean")  # or "zero", "forward", etc.
lagged_data = lagged_data.with_columns([
    pl.col(col).interpolate() for col in lagged_data.columns
])
# Drop constant columns
lagged_data = lagged_data.select([
    col for col in lagged_data.columns if lagged_data[col].unique().shape[0] > 1
])


In [ ]:
import pyarrow.dataset as ds

# Load the Parquet directory as a dataset
dataset = ds.dataset('../data/jane-street/lags.parquet', format="parquet")

# Convert to a Pandas DataFrame
lags = dataset.to_table().to_pandas()

# Inspect the data
print(lags.head())
print(lags.info())


In [ ]:
print(lags.columns)
print(lags.dtypes)
print(lags.isnull().sum())
print(lags.duplicated().sum())
import pandas as pd
train_data = pd.read_parquet('../data/XGBoost_data.parquet')

train_data = train_data.merge(lags, on=['date_id', 'time_id', 'symbol_id'], how='left')


In [ ]:
import pyarrow.dataset as ds
import pandas as pd

# Load the Parquet dataset
dataset = ds.dataset('../data/jane-street/lags.parquet', format="parquet")

# Convert to a Pandas DataFrame
lags = dataset.to_table().to_pandas()

# Convert `date_id` to a plain integer type (e.g., int16 or int32)
lags['date_id'] = lags['date_id'].astype('int16')

# Save the fixed version
lags.to_parquet('../data/jane-street/lags_fixed.parquet', index=False)


In [ ]:
fixed_lags = pd.read_parquet('../data/jane-street/lags_fixed.parquet')
print(fixed_lags.dtypes)


In [ ]:
train_data = pd.read_parquet('../data/XGBoost_data.parquet')
lags = pd.read_parquet('../data/jane-street/lags_fixed.parquet')

train_data = train_data.merge(lags, on=['date_id', 'time_id', 'symbol_id'], how='left')
